### Step 1—Set Up Your Environment

In [0]:
%python
import os

# Unset SPARK_HOME if it is set
if 'SPARK_HOME' in os.environ:
    del os.environ['SPARK_HOME']

# Install the package
%pip install databricks.automl-runtime

In [0]:
%restart_python

In [0]:
# Check if SPARK_HOME is set to a conflicting version
import os
if 'SPARK_HOME' in os.environ:
    del os.environ['SPARK_HOME']

# Install the databricks-automl-runtime package

%pip install databricks-feature-engineering

In [0]:
%restart_python

In [0]:
%restart_python

In [0]:
%python
import os

os.environ["MLFLOW_TRACKING_URI"] = "databricks"

In [0]:
%python
import mlflow
mlflow.set_tracking_uri("databricks")

In [0]:
# Retrieve the Databricks workspace URL
workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
print(workspace_url)

In [0]:
%python
from databricks.feature_store import FeatureStoreClient#from databricks.automl_runtime import Forecast
from databricks.automl_runtime import forecast
import mlflow

In [0]:
pip install databricks-automl-runtime

In [0]:
from databricks.automl import forecast

In [0]:
%python
from databricks.feature_store import FeatureStoreClient
from databricks.automl import forecast
import mlflow

# Set the MLflow model registry URI
mlflow.set_registry_uri("databricks")

# Load data from Unity Catalog
fs = FeatureStoreClient()
df = fs.read_table("kajal.forecast_schema.mofj_interest_rate_japan_feature")

# Run AutoML for forecasting
summary = forecast(
    dataset=df,
    target_col="Year_1",
    time_col="Time_period",
    horizon=30,
    frequency="d",
    timeout_minutes=120,
    primary_metric="smape"
)

display(summary)

In [0]:
%python
from databricks.feature_store import FeatureStoreClient
from databricks.automl import forecast
import mlflow

# Set the MLflow model registry URI
mlflow.set_registry_uri("databricks")

# Load data from Unity Catalog
fs = FeatureStoreClient()
df = fs.read_table("kajal.forecast_schema.mofj_interest_rate_japan_feature")

# Run AutoML for forecasting
summary = forecast(
    dataset=df,
    target_col="Year_1",
    time_col="Time_period",
    horizon=30,
    frequency="d",
    timeout_minutes=120,
    primary_metric="smape"
)

display(summary)

In [0]:
%python
from databricks.feature_store import FeatureStoreClient
from databricks.automl_runtime.forecast import Forecast

import mlflow

# Set the MLflow model registry URI
mlflow.set_registry_uri("databricks-mlflow://dbc-325dd65c-6716.cloud.databricks.com")

# Load data from Unity Catalog
fs = FeatureStoreClient()
df = fs.read_table("kajal.forecast_schema.mofj_interest_rate_japan_feature")

# Run AutoML for forecasting
summary = Forecast(
    df,
    target_col="Year_1",
    time_col="Time_period",
    horizon=30,
    frequency="d",
    timeout_minutes=120,
    primary_metric="mdape",
    output_database="default"
)

display(summary)

In [0]:
how to serve model as endpoint in databricks

In [0]:
import mlflow
import mlflow.pyfunc


# Log the model with mlflow
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model("forecast_model", python_model=ForecastModel())

# Deploy the model as an endpoint
model_uri = f"runs:/{run.info.run_id}/forecast_model"
endpoint_name = "forecast-model-endpoint"

# Create the endpoint
mlflow.endpoints.create(endpoint_name, model_uri)

In [0]:
import mlflow
import mlflow.pyfunc

# Define the forecast model class
class ForecastModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        import pandas as pd
        from statsmodels.tsa.holtwinters import ExponentialSmoothing
        
        self.model = ExponentialSmoothing

    def predict(self, context, model_input):
        model = self.model(model_input, seasonal='add', seasonal_periods=12).fit()
        forecast = model.forecast(steps=12)
        return forecast

# Log the model with mlflow
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model("forecast_model", python_model=ForecastModel())

# Deploy the model as an endpoint
model_uri = f"runs:/{run.info.run_id}/forecast_model"
endpoint_name = "forecast-model-endpoint"

# Create the endpoint
mlflow.endpoints.create(endpoint_name, model_uri)

In [0]:
pip install mlflow

In [0]:
%restart_python 

In [0]:
import mlflow.pyfunc

In [0]:
#import mlflow.pyfunc

# Load the registered model
model_name = "forecast_model_uc1"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

# Deploy the model as an endpoint
endpoint_name = "forecast-model-endpoint_uc1"
mlflow.endpoints.create(endpoint_name, model_uri)


In [0]:

from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")
endpoint = client.create_endpoint(
    name="forecast_endpoint_uc1",
    config={
        "served_entities": [
            {
                "name": "kajal",
                "entity_name": "kajal.forecast_schema.forecast_model_uc1",
                "entity_version": "3",
                "workload_size": "Small",
                "scale_to_zero_enabled": True
            }
        ],
        "traffic_config": {
            "routes": [
                {
                    "served_model_name": "kajal",
                    "traffic_percentage": 100
                }
            ]
        }
    }
)

In [0]:
%python
import mlflow.pyfunc
import mlflow

# Set the MLflow tracking URI to Databricks
mlflow.set_tracking_uri("databricks")

# Load the registered model
model_name = "forecast_model_uc1"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

# Deploy the model as an endpoint
endpoint_name = "forecast-model-endpoint_uc1"

# Create the endpoint using the Databricks REST API
import requests
import json

# Replace with your Databricks workspace URL and token
databricks_instance = "https://<your-databricks-instance>"
token = "<your-databricks-token>"

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

endpoint_url = f"{databricks_instance}/api/2.0/mlflow/endpoints/create"

data = {
    "name": endpoint_name,
    "config": {
        "served_models": [
            {
                "model_name": model_name,
                "model_version": model_version,
                "workload_size": "Small",
                "scale_to_zero_enabled": True
            }
        ]
    }
}

response = requests.post(endpoint_url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    print("Endpoint created successfully")
else:
    print(f"Failed to create endpoint: {response.text}")

In [0]:
import mlflow
import requests
import json

# Set the MLflow tracking URI to Databricks
mlflow.set_tracking_uri("databricks")

# Load the registered model
model_name = "forecast_model_uc1"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

# Deploy the model as an endpoint
endpoint_name = "forecast-model-endpoint_uc1"

# Use Databricks CLI to get the workspace URL
databricks_instance = "https://dbc-325dd65c-6716.cloud.databricks.com"

# Use Databricks CLI to get the access token
# Ensure you have Databricks CLI installed and configured
# You can configure it using `databricks configure --token`
headers = {
    "Authorization": f"Bearer {os.getenv('DATABRICKS_TOKEN')}",
    "Content-Type": "application/json"
}

endpoint_url = f"{databricks_instance}/api/2.0/mlflow/endpoints/create"

data = {
    "name": endpoint_name,
    "config": {
        "served_models": [
            {
                "model_name": model_name,
                "model_version": model_version,
                "workload_size": "Small",
                "scale_to_zero_enabled": True
            }
        ]
    }
}

response = requests.post(endpoint_url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    print("Endpoint created successfully")
else:
    print(f"Failed to create endpoint: {response.text}")

In [0]:
dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()

In [0]:
%python
import mlflow
import requests
import json
import os  # Import the os module

# Set the MLflow tracking URI to Databricks
mlflow.set_tracking_uri("databricks")

# Load the registered model
model_name = "forecast_model_uc1"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

# Deploy the model as an endpoint
endpoint_name = "forecast-model-endpoint_uc1"

# Use Databricks CLI to get the workspace URL
databricks_instance = "https://dbc-325dd65c-6716.cloud.databricks.com"

# Use Databricks CLI to get the access token
# Ensure you have Databricks CLI installed and configured
# You can configure it using `databricks configure --token`
headers = {
    "Authorization": f"Bearer {os.getenv('databricks_token')}",
    "Content-Type": "application/json"
}

endpoint_url = f"{databricks_instance}/api/2.0/mlflow/endpoints/create"

data = {
    "name": endpoint_name,
    "config": {
        "served_models": [
            {
                "model_name": model_name,
                "model_version": model_version,
                "workload_size": "Small",
                "scale_to_zero_enabled": True
            }
        ]
    }
}

response = requests.post(endpoint_url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    print("Endpoint created successfully")
else:
    print(f"Failed to create endpoint: {response.text}")

In [0]:
import os

# Retrieve the Databricks access token from environment variables
databricks_token = os.getenv('DATABRICKS_TOKEN')

# Display the token
display(databricks_token)

In [0]:
display(databricks_token)

In [0]:
import mlflow
from mlflow.deployments import get_deploy_client

# Set the MLflow tracking URI to Databricks
mlflow.set_tracking_uri("databricks")

# Load the registered model
model_name = "forecast_model_uc1"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

# Deploy the model as an endpoint
endpoint_name = "forecast-model-endpoint_uc1"

# Get the deploy client
client = get_deploy_client("databricks")
print(client)

# Create the endpoint
client.create_deployment(
    name=endpoint_name,
    model_uri=model_uri,
    config={
        "workload_size": "Small",
        "scale_to_zero_enabled": True
    }
)

print("Endpoint created successfully")

In [0]:
import mlflow

In [0]:
mlflow.register_model(
  "runs:/65139fb591d449199a6f348948cdd3bb/model", "kajal.forecast_schema.forecast_model_uc1"
)

In [0]:
model_uri = 'runs:/65139fb591d449199a6f348948cdd3bb/model'